In [2]:
import os
import numpy as np


import cv2
import dlib

from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
%matplotlib inline 

In [3]:
#detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('pretrained_detection_models/shape_predictor_68_face_landmarks.dat')
detector = cv2.CascadeClassifier('pretrained_detection_models/haarcascade_frontalface_default.xml')

In [4]:
TEST_PATH = 'data/train/attack/fixed/attack_highdef_client108_session01_highdef_video_controlled.mov'
PROTOCOLS = 'data/protocols'

In [5]:
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

class FaceAligner:
    def __init__(self, predictor, desired_left_eye=(0.35, 0.35),
        desired_face_width=256, desired_face_height=256):
        
        self.predictor = predictor
        self.desired_left_eye = desired_left_eye
        self.desired_face_width = desired_face_width
        self.desired_face_height = desired_face_height

    def align(self, image, gray, face_bbox):
        
        x, y, face_width, face_height = int(face_bbox[0]), int(face_bbox[1]), int(face_bbox[2]), int(face_bbox[3])
        shape = self.predictor(gray,dlib.rectangle(x, y, x + face_width, y + face_height))
        shape = shape_to_np(shape)

        # extract the left and right eye (x, y)-coordinates
        right_eye_points = shape[36:42]
        left_eye_points = shape[42:48]

        # compute the center of mass for each eye
        left_eye_center = left_eye_points.mean(axis=0).astype("int")
        right_eye_center = right_eye_points.mean(axis=0).astype("int")

        # compute the angle between the eye centroids
        dy = right_eye_center[1] - left_eye_center[1]
        dx = right_eye_center[0] - left_eye_center[0]
        angle = np.degrees(np.arctan2(dy, dx)) - 180

        # compute the desired right eye x-coordinate based on the
        # desired x-coordinate of the left eye
        desired_right_eye_x = 1.0 - self.desired_left_eye[0]

        # determine the scale of the new resulting image by taking
        # the ratio of the distance between eyes in the *current*
        # image to the ratio of distance between eyes in the
        # *desired* image
        distance = np.sqrt((dx ** 2) + (dy ** 2))
        desired_distance = (desired_right_eye_x - self.desired_left_eye[0])
        desired_distance *= self.desired_face_width
        scale = desired_distance / distance

        # compute center (x, y)-coordinates (i.e., the median point)
        # between the two eyes in the input image
        eyes_center = ((left_eye_center[0] + right_eye_center[0]) // 2,
            (left_eye_center[1] + right_eye_center[1]) // 2)

        # grab the rotation matrix for rotating and scaling the face
        M = cv2.getRotationMatrix2D( eyes_center, angle, scale)

        # update the translation component of the matrix
        tx = self.desired_face_width * 0.5
        ty = self.desired_face_height * self.desired_left_eye[1]
        M[0, 2] += (tx - eyes_center[0])
        M[1, 2] += (ty - eyes_center[1])

        # apply the affine transformation
        (w, h) = (self.desired_face_width, self.desired_face_height)
        aligned_face = cv2.warpAffine(image, M, (w, h),
            flags=cv2.INTER_CUBIC)

        # return the aligned face
        return aligned_face

In [6]:
face_aligner = FaceAligner(predictor, (0.31,0.31),224,224)
def crop_face(image):
    gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    detected = detector.detectMultiScale(gray_image, 1.1, 5)
    if len(detected) == 0:
        return None
    face_bbox = detector.detectMultiScale(gray_image, 1.1, 5)[0]
    #x, y, width, height = face_bbox[0], face_bbox[1], face_bbox[2], face_bbox[3]
    # return cv2.resize(image[y:y+height, x:x+width,:],(224,224))
    face_aligned = face_aligner.align(image, gray_image, face_bbox)
    return face_aligned

def extract_frames(video_path, out_path):
    count = 0
    capture = cv2.VideoCapture(video_path)
    is_captured, image = capture.read()
    while is_captured:
        face = crop_face(image)
        if face is not None:
            cv2.imwrite(out_path + os.path.basename(video_path)[:-4] + "_frame%d.jpg" % count,face) 
        capture.set(cv2.CAP_PROP_POS_MSEC,(count*500))
        is_captured, image = capture.read()
        count += 1
    capture.release()
    
def process_videos(video_pathes,out_path):
    for path in tqdm_notebook(video_pathes):
        extract_frames(path,out_path)

In [ ]:
%%time
video_pathes = []
with open(PROTOCOLS + '/real-test.txt') as file:
    video_pathes = file.read().split('\n')
    
print(len(video_pathes))

process_videos(video_pathes,'replay_attack_data_prepared/test/real/')

In [ ]:
%%time
video_pathes = []
with open(PROTOCOLS + '/attack-video-allsupports-test.txt') as file:
    video_pathes = file.read().split('\n')
    
print(len(video_pathes))

process_videos(video_pathes,'data/test/video_attack/')

In [ ]:
%%time
video_pathes = []
with open(PROTOCOLS + '/attack-photo-allsupports-test.txt') as file:
    video_pathes = file.read().split('\n')
    
print(len(video_pathes))

process_videos(video_pathes,'data/test/photo_attack/')

In [7]:
%%time
video_pathes = []
with open(PROTOCOLS + '/real-devel.txt') as file:
    video_pathes = file.read().split('\n')
    
print(len(video_pathes))

process_videos(video_pathes,'data/devel/real/')


video_pathes = []
with open(PROTOCOLS + '/attack-video-allsupports-devel.txt') as file:
    video_pathes = file.read().split('\n')
    
print(len(video_pathes))

process_videos(video_pathes,'data/devel/video_attack/')


video_pathes = []
with open(PROTOCOLS + '/attack-photo-allsupports-devel.txt') as file:
    video_pathes = file.read().split('\n')
    
print(len(video_pathes))

process_videos(video_pathes,'data/devel/photo_attack/')

60


HBox(children=(IntProgress(value=0, max=60), HTML(value='')))


120


HBox(children=(IntProgress(value=0, max=120), HTML(value='')))


180


HBox(children=(IntProgress(value=0, max=180), HTML(value='')))


Wall time: 17min 12s
